In [ ]:
import os
import json
import codecs

def safe_read_json(file_path):
    try:
        with codecs.open(file_path, 'r', 'utf-8-sig') as f:
            return json.load(f)
    except UnicodeDecodeError:
        with codecs.open(file_path, 'r', 'iso-8859-1') as f:
            return json.load(f)

def count_messages_in_file(file_path):
    try:
        messages = safe_read_json(file_path)
        if isinstance(messages, list):
            return len(messages)
        elif isinstance(messages, dict) and 'messages' in messages:
            return len(messages['messages'])
        return 0
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return 0

def count_messages(package_path):
    message_count = 0
    channel_counts = {}

    #Load the index.json file
    index_data = safe_read_json(os.path.join(package_path, 'index.json'))
    
    #Create a mapping of channel IDs to names
    channel_map = {str(k): v for k, v in index_data.items()}

    #Recursively search for message files
    for root, dirs, files in os.walk(package_path):
        for file in files:
            if file.endswith('.json') and file != 'index.json':
                file_path = os.path.join(root, file)
                count = count_messages_in_file(file_path)
                message_count += count

                #Determine channel name
                relative_path = os.path.relpath(root, package_path)
                channel_id = relative_path.split(os.path.sep)[0]
                if channel_id.startswith('c'):
                    channel_id = channel_id[1:]  #Remove the 'c' prefix
                channel_name = channel_map.get(channel_id, f'Channel {channel_id}')
                
                channel_counts[channel_name] = channel_counts.get(channel_name, 0) + count

    return message_count, channel_counts

if __name__ == "__main__":
    package_path = "./Downloads/package/messages"
    total_messages, channel_counts = count_messages(package_path)

    print("\n" + "="*50)
    print("Total Messages Sent:")
    print("="*50)
    print(total_messages)  # Only print the number

    print("\nMessages per Channel (sorted by count):")
    print("="*50)
    for channel, count in sorted(channel_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{channel}: {count}")  